# Responsible AI in Practice
## Building agents with context aware security
### Overview

This notebook demonstrates how to utilize Bedrock AgentCore to implement programmatic controls that protect controlled/sensitive information by dynamically managing tool access. We will build an AI Agent that:
1. Uses tool semantic search to discover the most relevant actions given a query.
2. Applies context-aware security strategies to determine whether the agent can access tools capable of leaking sensitive/controlled data.
3. Stores conversational memory tagged with classification (`public` or `controlled`) for fine-grained retrieval control.

By the end of this tutorial, you'll be able to:
- Invoke an agent that only uses tools appropriate for the current context.
- Manage memory retrieval based on tool classifications.
- Deploy an agent with Responsible AI practices to safeguard sensitive data.

This workflow is applicable across industries, and we have structured the notebook to allow for users to easily modify it for their use-cases.
**Prerequisites**:
> - An AWS account with Amazon Bedrock models as well as Bedrock AgentCore access enabled.
> - A Cognito User Pool to obtain OAuth tokens (`APPLICATION_USERPOOL_ID`, `APPLICATION_CLIENT_ID`, `APPLICATION_CLIENT_SECRET`)
> - Install required Python packages (handles below)

### Install required dependencies
We use `boto3`, `bedrock-agentcore-starter-toolkit`, and `strands-agents` for interacting with Amazon Bedrock models and AgentCore primatives. 

In [1]:
!pip install -U boto3 -q
!pip install bedrock-agentcore-starter-toolkit==0.1.14 -q
!pip install strands-agents -q
!pip install strands-agents-tools -q

### Import needed libraries

In [2]:
import json
import uuid
import boto3
import requests
import datetime

from strands import Agent
from strands.tools.mcp.mcp_client import MCPClient, MCPAgentTool
from mcp.client.streamable_http import streamablehttp_client
from mcp.types import Tool as MCPTool

### Configurations
These are the parameters for AgentCore Gateway as well as the client information for Cognito. You will have to fill these with your configuration values.

In [3]:
REGION = 'us-east-1'
GATEWAY_ID = 'mcp-tech-talk-travel-po9gnocfuh'
GATEWAY_URL = f'https://{GATEWAY_ID}.gateway.bedrock-agentcore.{REGION}.amazonaws.com/mcp'

# Cognito configuration for OAuth authentication
APPLICATION_USERPOOL_ID = ''
APPLICATION_CLIENT_ID = ''
APPLICATION_CLIENT_SECRET = ''
PERMISSION_SCOPE = ''

# Global token variable (set during initialization)
access_token = None

### Authentication functions
1. **get_oauth_token** - Get OAuth2 token from Cognito for authenticating with the MCP gateway.
2. **create_streamable_http_transport** - Create an HTTP transport for MCP communication with authentication.

In [4]:
def get_oauth_token(user_pool_id, client_id, client_secret, scope_string, region):
    """
    Get OAuth2 token from Cognito for authenticating with the MCP gateway.

    Uses client credentials flow to obtain an access token that will be used
    in the Authorization header for all gateway requests.

    Args:
        user_pool_id: Cognito user pool ID
        client_id: OAuth client ID
        client_secret: OAuth client secret
        scope_string: Space-separated list of OAuth scopes
        region: AWS region

    Returns:
        dict: Token response containing 'access_token', 'token_type', and 'expires_in'
    """
    # Cognito domain format requires user pool ID without underscore
    user_pool_id_without_underscore = user_pool_id.replace("_", "")
    token_url = f"https://{user_pool_id_without_underscore}.auth.{region}.amazoncognito.com/oauth2/token"

    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": scope_string,
    }

    response = requests.post(token_url, headers=headers, data=data)
    response.raise_for_status()  # Raise exception for HTTP errors
    return response.json()


def create_streamable_http_transport():
    """
    Create an HTTP transport for MCP communication with authentication.

    This transport is used by the MCP client to communicate with the gateway.
    The Authorization header includes the OAuth token obtained from Cognito.
    """
    global access_token
    if access_token is None:
        raise ValueError("Access token not initialized. Call get_oauth_token() first.")

    return streamablehttp_client(
        GATEWAY_URL,
        headers={'Authorization': f'Bearer {access_token}'}
    )

### Semantic tool search and filtering
1. **convert_mcp_tools_to_strands_format** - Convert MCP tool definitions to Strands Agent-compatible format.
2. **apply_tool_selection_strategy** - Apply a strategy for selecting which tools to provide to the agent.

The agent will use semantic search to find tools from a large toolbox. However, not all tools are allowed to execute in contexts that require handling sensitive data. After retrieving tools, we filter them:
- **Tool Classification**: Tools are categorized as `public` or `controlled`. When a controlled tool is requested, stricter security rules apply.
- **Tool Selection Strategies**:
  - **LeastRestrictiveTools**: If any public tool is available, allow only public tools and use public memory.
  - **MostRestrictiveTools**: If any controlled tool is needed, allow only controlled tools and use full memory.

In [5]:
def convert_mcp_tools_to_strands_format(mcp_client, search_results, top_n=5):
    """
    Convert MCP tool definitions to Strands Agent-compatible format.

    Takes the top N tools from search results and wraps them in MCPAgentTool
    objects that can be used by the Strands Agent.

    Args:
        mcp_client: Active MCP client instance
        search_results: List of tool search results from gateway
        top_n: Number of top results to convert

    Returns:
        dict: Map of tool name to MCPAgentTool instance
    """
    strands_tools = {}

    for tool in search_results[:top_n]:
        # Create MCP tool definition
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )

        # Wrap in Strands-compatible format
        strands_tools[tool["name"]] = MCPAgentTool(mcp_tool, mcp_client)

    return strands_tools


def apply_tool_selection_strategy(tool_list, strategy='LeastRestrictiveTools'):
    """
    Apply a strategy for selecting which tools to provide to the agent.

    This implements the core access control logic by determining:
    1. Whether to filter tools by classification (public vs controlled)
    2. What memory filter to apply when retrieving conversation history

    Strategies:
    - LeastRestrictiveTools: If ANY public tool is selected, switch to public-only mode
      and filter out controlled tools. Use public conversation history.

    - MostRestrictiveTools: If ANY controlled tool is selected, use only controlled tools.
      Use full conversation history (no filter).
      If only public tools, use public-only mode.

    - ModelJudge: (Not implemented) Let the model decide based on query analysis

    Args:
        tool_list: List of tool names that were selected by search
        strategy: Strategy name to apply

    Returns:
        tuple: (metadata_filter, filtered_tool_list, classification)
            - metadata_filter: Filter for memory retrieval (or None for no filter)
            - filtered_tool_list: Potentially filtered list of tool names
            - classification: 'public' or 'controlled' for memory storage
    """
    metadata_filter = None
    filtered_tools = tool_list
    classification = None

    if strategy == 'LeastRestrictiveTools':
        # Strategy: If any public tool is present, switch to public-only mode
        has_public_tool = any('public' in tool_name for tool_name in tool_list)

        if has_public_tool:
            # Use only public tools and public conversation history
            filtered_tools = [t for t in tool_list if 'public' in t]
            classification = 'public'
            metadata_filter = [{
                'left': {'metadataKey': 'classification'},
                'operator': 'EQUALS_TO',
                'right': {'metadataValue': {'stringValue': 'public'}}
            }]
        else:
            # All tools are controlled, use full history
            classification = 'controlled'
            # No filter - retrieve all conversation history

    elif strategy == 'MostRestrictiveTools':
        # Strategy: If any controlled tool is present, use only controlled tools
        has_controlled_tool = any('controlled' in tool_name for tool_name in tool_list)

        if has_controlled_tool:
            # Keep all tools (controlled and public) and use full history
            filtered_tools = [t for t in tool_list if 'controlled' in t]
            classification = 'controlled'
            # No filter - retrieve all conversation history
        else:
            # Only public tools available
            classification = 'public'
            metadata_filter = [{
                'left': {'metadataKey': 'classification'},
                'operator': 'EQUALS_TO',
                'right': {'metadataValue': {'stringValue': 'public'}}
            }]

    elif strategy == 'ModelJudge':
        # TODO: Implement model-based decision making
        # Could use a separate LLM call to classify the query and determine
        # whether controlled data access is necessary
        raise NotImplementedError("ModelJudge strategy is not yet implemented")

    else:
        raise ValueError(f"Unknown strategy: {strategy}")

    return metadata_filter, filtered_tools, classification

### Session management functions using AgentCore Memory
1. **get_conversation_history** - Retrieve conversation history from Bedrock AgentCore Memory.
2. **add_message_to_memory** - Store a conversation message in Bedrock AgentCore Memory.

Every message stored in memory has a metadata tag indicating its classification (`public` or `controlled`). This enables the agent to:
- Retrieve only memory that matches the current context (filtering out inappropriate cross-context leakage).
- Maintain a secure dialogue history when switching between contexts.

In [6]:
def get_conversation_history(memory_id, actor_id, session_id, tool_names, strategy='LeastRestrictiveTools'):
    """
    Retrieve conversation history from Bedrock AgentCore Memory.

    This function:
    1. Applies the tool selection strategy to determine memory filtering
    2. Retrieves conversation events from the memory store
    3. Converts events to Strands message format
    4. Returns filtered tools and classification for new messages

    Args:
        memory_id: ID of the memory store
        actor_id: ID of the user/actor
        session_id: ID of the conversation session
        tool_names: List of tool names that were selected
        strategy: Tool selection strategy to apply

    Returns:
        tuple: (messages, filtered_tools, classification)
            - messages: List of conversation messages in Strands format
            - filtered_tools: Tool names after applying strategy
            - classification: Classification for storing new messages
    """
    memory_client = boto3.client('bedrock-agentcore', region_name=REGION)

    # Apply strategy to determine filtering
    metadata_filter, filtered_tools, classification = apply_tool_selection_strategy(
        tool_list=tool_names,
        strategy=strategy
    )

    # Build parameters for memory retrieval
    parameters = {
        'memoryId': memory_id,
        'actorId': actor_id,
        'sessionId': session_id,
        'includePayloads': True,
        'maxResults': 100  # Retrieve last 100 conversation turns
    }

    # Apply metadata filter if strategy determined one is needed
    if metadata_filter is not None:
        parameters['filter'] = {'eventMetadata': metadata_filter}

    # Retrieve conversation events
    response = memory_client.list_events(**parameters)

    # Convert events to messages with timestamps
    messages = []
    for event in response.get('events', []):
        payload = event.get('payload', [{}])[0]
        if 'conversational' in payload:
            messages.append({
                'timestamp': event.get('eventTimestamp').timestamp(),
                'role': payload['conversational']['role'].lower(),
                'text': payload['conversational']['content']['text']
            })

    # Sort by timestamp (oldest first)
    messages.sort(key=lambda x: x['timestamp'])

    # Convert to Strands message format
    strands_messages = []
    for msg in messages:
        # Map role names (handle any unexpected roles as 'assistant')
        role = msg['role'] if msg['role'] in ('user', 'assistant') else 'assistant'

        strands_messages.append({
            'role': role,
            'content': [{'text': msg['text']}]
        })

    return strands_messages, filtered_tools, classification


def add_message_to_memory(memory_id, actor_id, session_id, role, classification, text):
    """
    Store a conversation message in Bedrock AgentCore Memory.

    Messages are tagged with:
    - refId: Unique identifier for the message
    - classification: 'public' or 'controlled' for filtering

    Args:
        memory_id: ID of the memory store
        actor_id: ID of the user/actor
        session_id: ID of the conversation session
        role: Message role ('USER' or 'ASSISTANT')
        classification: Message classification ('public' or 'controlled')
        text: Message content
    """
    memory_client = boto3.client('bedrock-agentcore', region_name=REGION)

    # Create metadata for the message
    metadata = {
        'refId': {'stringValue': str(uuid.uuid4())},
        'classification': {'stringValue': classification}
    }

    # Store the message
    parameters = {
        'memoryId': memory_id,
        'actorId': actor_id,
        'sessionId': session_id,
        'eventTimestamp': datetime.datetime.now(),
        'payload': [{
            'conversational': {
                'content': {'text': text},
                'role': role
            }
        }],
        'metadata': metadata,
    }

    response = memory_client.create_event(**parameters)
    return response

### Main agent invocation "loop"

In [7]:
def invoke(payload=None, context=None):
    """
    Main entry point for invoking the travel planning agent.

    This function orchestrates the complete agent workflow:
    1. Extract parameters from payload
    2. Search for relevant tools using semantic search
    3. Apply tool selection strategy
    4. Retrieve conversation history with appropriate filtering
    5. Invoke agent with filtered tools and history
    6. Store the conversation turn in memory

    Args:
        payload: Dict containing:
            - prompt: User's query/request
            - memory_id: ID of the conversation memory store
            - actor_id: ID of the user
            - session_id: ID of the conversation session
            - strategy: Tool selection strategy ('LeastRestrictiveTools' or 'MostRestrictiveTools')
            - topk: Number of tools to retrieve from search
            - model_id: Bedrock model ID to use
            - system_prompt_file: Path to system prompt file (optional)
        context: Execution context (unused, for Lambda compatibility)

    Returns:
        dict: Agent response containing the answer and any tool calls made
    """
    if payload is None:
        payload = {}

    # ========================================================================
    # 1. EXTRACT PARAMETERS
    # ========================================================================
    prompt = payload.get('prompt')
    memory_id = payload.get('memory_id')
    actor_id = payload.get('actor_id', 'user_1')
    session_id = payload.get('session_id', 'session_test_1')
    strategy = payload.get('strategy', 'LeastRestrictiveTools')
    top_k = payload.get('topk', 10)
    model_id = payload.get('model_id', 'global.anthropic.claude-sonnet-4-20250514-v1:0')
    system_prompt_file = payload.get('system_prompt_file', 'travel_agent_system.txt')
    display_check = payload.get('display', '')

    # Load and customize system prompt
    with open(system_prompt_file, 'r') as f:
        system_prompt = f.read()

    # Inject current date and actor ID into system prompt
    current_date = datetime.date.today()
    day_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"][current_date.weekday()]
    system_prompt = (system_prompt
                    .replace('{{CURRENT_DATE}}', f'{current_date} ({day_of_week})')
                    .replace('{{ACTOR_ID}}', actor_id))

    # ========================================================================
    # 2. SEARCH FOR RELEVANT TOOLS
    # ========================================================================
    mcp_client = MCPClient(create_streamable_http_transport)
    with mcp_client:
        all_tools = mcp_client.list_tools_sync()
        search_agent = Agent(tools=all_tools)
        if 'ALL_TOOLS' in display_check:
            return {'statusCode': 201, 'all_tools': search_agent.tool_names}

        # Use the gateway's semantic search capability
        # This returns the most relevant tools based on the prompt
        search_result = search_agent.tool.x_amz_bedrock_agentcore_search(query=prompt)
        search_response = json.loads(search_result["content"][0]["text"])
        tool_search_results = search_response["tools"]
        available_tools = convert_mcp_tools_to_strands_format(mcp_client, tool_search_results, top_k)

        # ====================================================================
        # 3. APPLY TOOL SELECTION STRATEGY
        # ====================================================================
        conversation_history, filtered_tool_names, message_classification = get_conversation_history(
            memory_id=memory_id,
            actor_id=actor_id,
            session_id=session_id,
            tool_names=list(available_tools.keys()),
            strategy=strategy
        )
        filtered_tools = [available_tools[name] for name in filtered_tool_names]

        # ====================================================================
        # 4. INVOKE AGENT
        # ====================================================================
        agent = Agent(system_prompt=system_prompt, model=model_id, tools=filtered_tools)
        agent.messages = conversation_history
        response = agent(prompt)

        # ====================================================================
        # 5. STORE CONVERSATION
        # ====================================================================
        add_message_to_memory(
            memory_id=memory_id,
            actor_id=actor_id,
            session_id=session_id,
            role='USER',
            classification=message_classification,
            text=prompt
        )

        add_message_to_memory(
            memory_id=memory_id,
            actor_id=actor_id,
            session_id=session_id,
            role='ASSISTANT',
            classification=message_classification,
            text=str(response)
        )

        return {
            'statusCode': 200,
            'response': response,
            'classification': message_classification,
            'tools_used': filtered_tool_names
        }

## Testing the workflow

In [16]:
# Obtain OAuth token
access_token = get_oauth_token(
    APPLICATION_USERPOOL_ID,
    APPLICATION_CLIENT_ID,
    APPLICATION_CLIENT_SECRET,
    PERMISSION_SCOPE,
    REGION
).get('access_token')

# AgentCore Memory parameters
memory_id = 'TieredDataMemoryStore-GB7Pl2ECX3'
actor_id = 'chaeclrk'
session_id = 'tech-demo-weekend-getaway'
response = invoke({'display': 'ALL_TOOLS'})
print('================\nAll Tools:\n================')
_ = [print(f'{str(i+1).zfill(2)}. {t}') for i, t in enumerate(response.get('all_tools'))]

All Tools:
01. x_amz_bedrock_agentcore_search
02. TravelTools___calculate_trip_cost__public
03. TravelTools___check_visa_requirements__public
04. TravelTools___find_attractions__public
05. TravelTools___find_travel_insurance__public
06. TravelTools___get_airport_info__public
07. TravelTools___get_currency_exchange__public
08. TravelTools___get_destination_weather__public
09. TravelTools___get_local_transportation__public
10. TravelTools___get_loyalty_status__controlled
11. TravelTools___get_my_bookings__controlled
12. TravelTools___get_packing_checklist__public
13. TravelTools___get_past_trips__controlled
14. TravelTools___get_saved_payment_methods__controlled
15. TravelTools___get_saved_travelers__controlled
16. TravelTools___get_travel_alerts__public
17. TravelTools___get_travel_credits__controlled
18. TravelTools___get_travel_preferences__controlled
19. TravelTools___get_upcoming_trips__controlled
20. TravelTools___request_travel_approval__controlled
21. TravelTools___search_car_ren

In [9]:
prompt1 = 'I want to plan a quick weekend trip near San Francisco. What are some good options within 3 hours?'
payload = {
    'prompt': prompt1,
    'memory_id': memory_id,
    'actor_id': actor_id,
    'session_id': session_id
}
response1 = invoke(payload)
print(f'''
\n\n
=========================
Classification and Tools:
=========================
Classification:  {response1.get("classification")}
Tools Available: {response1.get("tools_used")}
''')

I'd be happy to help you plan a weekend trip near San Francisco! However, I don't currently have access to tools that can search for destinations or attractions within a specific travel radius. 

To give you the best recommendations, could you let me know a few specific cities or areas you're considering? For example, are you thinking about places like Napa Valley, Monterey, Santa Barbara, Lake Tahoe, or any other particular destinations? Once you mention some specific locations, I can help you find flights, restaurants, and check any travel information for those areas.

Alternatively, if you have a particular type of experience in mind (wine country, beaches, mountains, etc.), that would help me understand what kind of destination search tools might be most useful for your planning.



Classification and Tools:
Classification:  public
Tools Available: ['TravelTools___search_flights__public', 'TravelTools___search_restaurants__public', 'TravelTools___get_travel_alerts__public', 'Travel

In [10]:
prompt2 = 'Can you search for a hotel room for me and my wife for a single night on Friday? I want to travel this next weekend. Also, wine country sounds fun.'
payload = {
    'prompt': prompt2,
    'memory_id': memory_id,
    'actor_id': actor_id,
    'session_id': session_id
}
response2 = invoke(payload)
print(f'''
\n\n
=========================
Classification and Tools:
=========================
Classification:  {response2.get("classification")}
Tools Available: {response2.get("tools_used")}
''')

I'd be happy to help you find a hotel in wine country for this weekend! Since you mentioned wanting to go this Friday, I'll search for November 21st (this Friday) with checkout on Saturday, November 22nd.

For wine country near San Francisco, I'll search in Napa Valley, which is one of the most popular wine regions within about an hour and a half from the city.
Tool #1: TravelTools___search_hotels__public
Great news! I found several excellent hotel options in Napa Valley for Friday night, November 21st. Here are the top choices:

**Westin Inn & Suites** stands out with a 4.9 rating and $250/night for a suite. It includes breakfast, spa access, and free cancellation until 24 hours before your stay.

**InterContinental Hotel** offers a deluxe queen room for $270/night with a 4.8 rating and free cancellation up to 48 hours before. It has spa and gym facilities.

**Hyatt Inn & Suites** provides good value at $216/night for a double queen room with a 4.5 rating, though it's non-refundable.


In [11]:
prompt3 = 'Hmmm... Look at my previous bookings to see if I have stayed at any of these before.'
payload = {
    'prompt': prompt3,
    'memory_id': memory_id,
    'actor_id': actor_id,
    'session_id': session_id,
    'topk': 2,
}
response3 = invoke(payload)
print(f'''
\n\n
=========================
Classification and Tools:
=========================
Classification:  {response3.get("classification")}
Tools Available: {response3.get("tools_used")}
''')


Tool #1: TravelTools___get_my_bookings__controlled
Looking at your booking history, I don't see any previous stays at the Napa Valley hotels I mentioned (Westin Inn & Suites, InterContinental Hotel, Hyatt Inn & Suites, or InterContinental Grand Hotel). Your recent hotel booking was at the Marriott Marquis Times Square in New York.

This means you'd be experiencing any of these Napa Valley properties for the first time! Given that, I'd recommend the **Westin Inn & Suites** for its excellent 4.9 rating, included breakfast, and spa access at $250/night. It offers great value and amenities perfect for a wine country weekend.

Would you like me to help you with anything else for your trip, such as finding restaurants or checking the weather forecast for the weekend?



Classification and Tools:
Classification:  controlled
Tools Available: ['TravelTools___get_my_bookings__controlled', 'TravelTools___get_past_trips__controlled']



In [12]:
prompt4 = 'I need to update my travel profile first. What do you need from me?'
payload = {
    'prompt': prompt4,
    'memory_id': memory_id,
    'actor_id': actor_id,
    'session_id': session_id,
    'topk': 1,
}
response4 = invoke(payload)
print(f'''
\n\n
=========================
Classification and Tools:
=========================
Classification:  {response4.get("classification")}
Tools Available: {response4.get("tools_used")}
''')

I can help you update your traveler profile! To do this, I'll need your traveler ID first. 

Once I have that, I can update any of the following information if needed:
- First and last name
- Email address
- Phone number
- Passport number and expiry date

What's your traveler ID, and which details would you like to update?



Classification and Tools:
Classification:  controlled
Tools Available: ['TravelTools___update_traveler_profile__controlled']



In [13]:
prompt5 = 'I need to update my travel profile. My travel ID is KTN12345678. My name is John Smith (john.smith@email.com) and my number is 555-0123. Passport ID: US1234567 expies 2028-06-15.'
payload = {
    'prompt': prompt5,
    'memory_id': memory_id,
    'actor_id': actor_id,
    'session_id': session_id,
    'topk': 1,
}
response5 = invoke(payload)
print(f'''
\n\n
=========================
Classification and Tools:
=========================
Classification:  {response5.get("classification")}
Tools Available: {response5.get("tools_used")}
''')


Tool #1: TravelTools___update_traveler_profile__controlled
Perfect! I've successfully updated your traveler profile. All your information has been updated including your name (John Smith), email (john.smith@email.com), phone number (555-0123), and passport details (US1234567 expiring 2028-06-15).

Now that your profile is current, would you like to proceed with booking one of those Napa Valley hotels for your weekend wine country trip? I still recommend the Westin Inn & Suites for Friday night at $250 with its excellent amenities and ratings.



Classification and Tools:
Classification:  controlled
Tools Available: ['TravelTools___update_traveler_profile__controlled']



In [14]:
prompt6 = 'Okay. Now find me some good flights. Also, what\'s my passport number so I know it\'s correct?'
payload = {
    'prompt': prompt6,
    'memory_id': memory_id,
    'actor_id': actor_id,
    'session_id': session_id,
    #'topk': 1,
}
response6 = invoke(payload)
print(f'''
\n\n
=========================
Classification and Tools:
=========================
Classification:  {response6.get("classification")}
Tools Available: {response6.get("tools_used")}
''')

I'd be happy to help you find flights to Napa Valley! To search for the best options, I need a few details:

**Which airport would you prefer to fly from?** San Francisco area has several options like SFO (San Francisco International), OAK (Oakland), or SJC (San Jose).

**What time do you prefer to depart on Friday?** Morning, afternoon, or evening flights?

**Do you need a return flight for Sunday?** If so, what time would work best for your return?

Regarding your passport number, I don't have access to your personal documents or identification information. You'll need to check your physical passport or any digital copies you may have saved. For domestic travel within California, you'll only need a valid driver's license or state ID anyway, so your passport isn't required for this wine country trip.

Once you provide the flight details, I can search for the best options and pricing for your weekend getaway!



Classification and Tools:
Classification:  public
Tools Available: ['Trave

In [15]:
prompt7 = 'What is my travelers info?'
payload = {
    'prompt': prompt7,
    'memory_id': memory_id,
    'actor_id': actor_id,
    'session_id': session_id,
    'topk': 2,
}
response7 = invoke(payload)
print(f'''
\n\n
=========================
Classification and Tools:
=========================
Classification:  {response7.get("classification")}
Tools Available: {response7.get("tools_used")}
''')


Tool #1: TravelTools___get_saved_travelers__controlled
Based on your saved traveler information, here are your details:

**Your Profile (Primary):**
- Name: John Smith
- Email: john.smith@email.com
- Phone: +1-555-0123
- Passport: US1234567 (expires 2028-06-15)
- Known Traveler Number: KTN12345678
- Global Entry: Yes

**Your Companion Profile:**
- Name: Jane Smith
- Email: jane.smith@email.com
- Phone: +1-555-0124
- Passport: US7654321 (expires 2027-03-22)

Since you mentioned traveling with your wife, I assume the companion profile is for her. You're all set with valid passports and you have Global Entry status which will help with security lines! 

Now for those flight details - which departure airport works best for you, and what time preferences do you have for Friday's departure?



Classification and Tools:
Classification:  controlled
Tools Available: ['TravelTools___get_saved_travelers__controlled', 'TravelTools___update_traveler_profile__controlled']

